#### Tarea 3: Machine Learning
### Universidad de Valparaíso
## Alumno: Bastián Barraza
## Profesor: Alejandro Pereira

In [57]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.svm import SVR
import sklearn.metrics as sk_metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import cross_val_score
from scipy.special import expit
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/bastianbm7/Machine-Learning-Course/main/Data/data.csv")

### Comparar regression logística, SVM, Naïve Bayes, LDA y QDA usando las métricas de desmpeño correspondientes. Hacer un código con comentarios y analysis.

#### Limpieza de datos:

Primero se realiza la limpieza de datos. Para garantizar la limpieza se abordarán los siguientes puntos:

1) Renombrar columnas.
2) Reemplazar valores mal ingresados.
3) Eliminar columnas que no se utilizarán.
4) Eliminar valores perdidos.

In [3]:
df.rename(columns = {'num       ': 'attack'}, inplace = True) 
df.replace("?", np.nan, inplace = True)
df.drop(columns = ['slope', 'ca', 'thal'], inplace = True)
df.dropna(inplace= True)
df.shape

(261, 11)

In [4]:
# Valores perdidos:
df.isnull().sum()

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
attack      0
dtype: int64

In [5]:
df['exang'].unique()

array(['0', '1'], dtype=object)

Notamos que no existen luego de realizar la limpieza, no existen datos perdidos y todos los datos tienen una estructura consistente. 

#### Definir variables

Ahora, se busca separar nuestro variable de respuesta de las demás covariables. En este caso, la variable de respuesta es la presencia de angina luego de realizar ejercisio físico (denotada como exang en el conjunto de datos).

In [6]:
y = df['exang']
x = df.drop(['exang'], axis=1)

#### Separar variables de prueba y entrenamiento:

Luego, serparamos el conjunto de datos en 80% entrenamiento y 20% prueba.

In [7]:
random_seed = 1
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

#### 1.- Regresion Logística:

In [8]:
model = LogisticRegression()
model.fit(x_train, y_train)

C:\Users\Bastian Barraza M\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [11]:
y_pred = model.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print("Precisión del modelo:", accuracy)

Precisión del modelo: 0.8113207547169812


In [12]:
confusion_matrix(y_test, y_pred)

array([[32,  5],
       [ 5, 11]], dtype=int64)

#### 2.- SVM: 

In [37]:
model = SVR(kernel='linear')
model.fit(x_train, y_train)

SVR(kernel='linear')

In [38]:
y_pred = model.predict(x_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("MSE:", mse)
print("R²:", r2)

MSE: 0.1463920195807335
R²: 0.305379758442094


#### Medir el rendimiento con Cross-validation

In [15]:
#scores = cross_val_score(model, x_train, y_train, cv=5)

In [39]:
scores

array([0.35375425, 0.75884766, 0.37339859, 0.42852992, 0.41139044])

In [40]:
scores.mean()

0.46518417226372427

In [41]:
scores.std()

0.14920909251234324

#### 3.- Naive Bayes:

In [21]:
model = GaussianNB()
model.fit(x_train, y_train)

GaussianNB()

In [23]:
# Predecir la variable respuesta (y) con los datos de prueba
y_pred = model.predict(x_test)

# Evaluar el modelo usando accuracy score
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy score: ", accuracy)

Accuracy score:  0.8490566037735849


#### 4.- LDA

In [47]:
lda = LinearDiscriminantAnalysis()
lda.fit(x_train, y_train)


In [48]:
y_pred = lda.predict(x_test)

print('Accuracy:', accuracy_score(y_test, y_pred))
print('Confusion matrix:\n', confusion_matrix(y_test, y_pred))

Accuracy: 0.8301886792452831
Confusion matrix:
 [[32  5]
 [ 4 12]]


#### 5.- QDA

In [51]:
qda = QuadraticDiscriminantAnalysis()
qda.fit(x_train, y_train)

QuadraticDiscriminantAnalysis()

In [58]:
Y_pred = qda.predict(x_test)

conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print('Matriz de confusión:')
print(conf_matrix)

print('Informe de clasificación:')
print(class_report)

Matriz de confusión:
[[32  5]
 [ 4 12]]
Informe de clasificación:
              precision    recall  f1-score   support

           0       0.89      0.86      0.88        37
           1       0.71      0.75      0.73        16

    accuracy                           0.83        53
   macro avg       0.80      0.81      0.80        53
weighted avg       0.83      0.83      0.83        53

